# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [158]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-527f536624-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Movie Workflow Series ("Tv series Without a Trace explorative search") 


Consider the following exploratory scenario:


> we are interested in the TV series "Without a Trace" and we want to investigate the main aspects related to the actors and directors involved in the production, know the numerber of seasons and check what are the episodes which got the higher success/impact.


## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P4969`    | derivative work      | predicate |
| `wd:Q826477` | Without a Trace    | node |
| `wd:Q733960` | Cold Case       | node |



Also consider

```
wd:Q826477 ?p ?obj .
```

is the BGP to retrieve all **properties of Without a Trace**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.



The workload should

1. Identify the BGP for television series

2. Return the number of seasons and episodes per season of the tv series (the result set must be triples of season IRI, label and #episodes).

3. Get the number of episodes in which the cast members played a role. Who are the most present actors? (the result set must be a list of triples actor/actress IRI, label and #episodes)

4. Check who is the actor who acted in more films while working on "Without a Trace" (the result set must be a list of triples actor/actress IRI, label and #films).

5. Compare Without a Trace with the tv series "Cold Case" in terms of number of seasons, episods and cast members (the result set must be two elements -one for each tv series- of tv series IRI, label, #seasons, #episodes and #cast members).

6. Return the actors who are members of the cast of Without a Trace have [Kavin Bacon number](https://en.wikipedia.org/wiki/Six_Degrees_of_Kevin_Bacon#:~:text=Kevin%20Bacon%20himself%20has%20a,Bacon%20number%20is%20N%2B1.) equal to 2 (the result set must be a list of couples actor/actress IRI and label).

7. Consider the actors who are members of the cast of Cold Case. Amongst the tv series which these actors acted return only those which received more than 2 awards (the result set must be triples of tv series IRI, label, #awards won).

## Task 1
Identify the BGP for television series

In [160]:
# query example
queryString = """
SELECT DISTINCT ?obj ?label
WHERE {
   # bind something
   wd:Q826477 wdt:P31 ?obj .
   # get the label
   ?obj sc:name ?label.
}
LIMIT 20

"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q5398426'), ('label', 'television series')]
1


Final query for this task

In [161]:
queryString = """
SELECT DISTINCT ?p ?label
WHERE {
   # BGP for television series
   wd:Q5398426 ?p ?obj .
   
   # Get the label
   ?p sc:name ?label.
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P9374'), ('label', 'ODLIS ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('label', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('label', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1709'), ('label', 'equivalent class')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('label', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('label', 'properties for this type')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('label', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2670'), ('label', 'has parts of the class')]
[('p', 'http://www.wikidata.org/prop/direct/P2671'), ('label', 'Google Knowledge Graph ID')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('label', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P3219'), ('label', 'Encyclopædia Universalis ID')]
[('p', 'http://www.wikidata.org/prop/direct

## Task 2
Return the number of seasons and episodes per season of the tv series (the result set must be triples of season IRI, label and #episodes).

In [166]:
queryString = """
SELECT DISTINCT ?p ?property ?label
WHERE {

     wd:Q826477 ?p ?obj.
     ?p sc:name ?property.
     ?obj sc:name ?label.

  
} LIMIT 50


"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P750'), ('property', 'distributed by'), ('label', 'Warner Bros. Television Studios')]
[('p', 'http://www.wikidata.org/prop/direct/P449'), ('property', 'original broadcaster'), ('label', 'CBS')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('property', 'cast member'), ('label', 'Anthony LaPaglia')]
[('p', 'http://www.wikidata.org/prop/direct/P364'), ('property', 'original language of film or TV show'), ('label', 'English')]
[('p', 'http://www.wikidata.org/prop/direct/P495'), ('property', 'country of origin'), ('label', 'United States of America')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('property', 'instance of'), ('label', 'television series')]
[('p', 'http://www.wikidata.org/prop/direct/P840'), ('property', 'narrative location'), ('label', 'New York City')]
[('p', 'http://www.wikidata.org/prop/direct/P2061'), ('property', 'aspect ratio'), ('label', '16:9 aspect ratio')]
[('p', 'http://www.wikidata.org/prop/direct/P

Final query for this task

In [169]:
queryString = """
SELECT DISTINCT ?IRI ?label (count(?ep) AS ?episodes)
WHERE {
 
 # Get the IRI and label of seasons
 wd:Q826477 wdt:P527 ?IRI.
 ?IRI sc:name ?label.
 
 # Get the number of episodes
 ?IRI wdt:P527 ?ep.
 
  
}GROUP BY ?label ?IRI
"""

print("Results")
x=run_query(queryString)

Results
[('IRI', 'http://www.wikidata.org/entity/Q1120248'), ('label', 'Without a Trace, season 1'), ('episodes', '23')]
[('IRI', 'http://www.wikidata.org/entity/Q3729811'), ('label', 'Without a Trace, season 5'), ('episodes', '24')]
[('IRI', 'http://www.wikidata.org/entity/Q3729815'), ('label', 'Without a Trace, season 6'), ('episodes', '18')]
[('IRI', 'http://www.wikidata.org/entity/Q3729810'), ('label', 'Without a Trace, season 4'), ('episodes', '24')]
[('IRI', 'http://www.wikidata.org/entity/Q3729817'), ('label', 'Without a Trace, season 3'), ('episodes', '23')]
[('IRI', 'http://www.wikidata.org/entity/Q3729816'), ('label', 'Without a Trace, season 7'), ('episodes', '24')]
[('IRI', 'http://www.wikidata.org/entity/Q3729812'), ('label', 'Without a Trace, season 2'), ('episodes', '24')]
7


## Task 3
Get the number of episodes in which the cast members played a role. Who are the most present actors? (the result set must be a list of triples actor/actress IRI, label and #episodes)

In [172]:
queryString = """
SELECT ?cast ?name 
WHERE {

  wd:Q826477 wdt:P527 ?obj.
  #?obj sc:name ?season.
  ?obj wdt:P527 ?ep.
  ?ep wdt:P161 ?cast.
  ?cast sc:name ?name.
  #wd:Q1120248 wdt:P527 ?ep.
  #?ep wdt:P161 ?cast.
} ORDER BY DESC(?numEpisodes) LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('cast', 'http://www.wikidata.org/entity/Q373989'), ('name', 'David Paymer')]
[('cast', 'http://www.wikidata.org/entity/Q139341'), ('name', 'Zach Grenier')]
[('cast', 'http://www.wikidata.org/entity/Q3265830'), ('name', 'Lucille Soong')]
[('cast', 'http://www.wikidata.org/entity/Q430872'), ('name', 'Marianne Jean-Baptiste')]
[('cast', 'http://www.wikidata.org/entity/Q430872'), ('name', 'Marianne Jean-Baptiste')]
[('cast', 'http://www.wikidata.org/entity/Q430872'), ('name', 'Marianne Jean-Baptiste')]
[('cast', 'http://www.wikidata.org/entity/Q430872'), ('name', 'Marianne Jean-Baptiste')]
[('cast', 'http://www.wikidata.org/entity/Q430872'), ('name', 'Marianne Jean-Baptiste')]
[('cast', 'http://www.wikidata.org/entity/Q430872'), ('name', 'Marianne Jean-Baptiste')]
[('cast', 'http://www.wikidata.org/entity/Q430872'), ('name', 'Marianne Jean-Baptiste')]
[('cast', 'http://www.wikidata.org/entity/Q430872'), ('name', 'Marianne Jean-Baptiste')]
[('cast', 'http://www.wikidata.org/entity

Final query for this task

In [170]:
queryString = """
SELECT ?cast ?name (COUNT(?name) AS ?numEpisodes)
WHERE {
 
  # Get the season and episodes 
  wd:Q826477 wdt:P527 ?obj.
  #?obj sc:name ?season.
  ?obj wdt:P527 ?ep.
  
  # Get the actor/actress IRI and label, then count
  ?ep wdt:P161 ?cast.
  ?cast sc:name ?name.
  
  
} GROUP BY ?cast ?name ORDER BY DESC(?numEpisodes) LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('cast', 'http://www.wikidata.org/entity/Q308124'), ('name', 'Anthony LaPaglia'), ('numEpisodes', '160')]
[('cast', 'http://www.wikidata.org/entity/Q235075'), ('name', 'Poppy Montgomery'), ('numEpisodes', '160')]
[('cast', 'http://www.wikidata.org/entity/Q503040'), ('name', 'Eric Close'), ('numEpisodes', '160')]
[('cast', 'http://www.wikidata.org/entity/Q551608'), ('name', 'Enrique Murciano'), ('numEpisodes', '160')]
[('cast', 'http://www.wikidata.org/entity/Q430872'), ('name', 'Marianne Jean-Baptiste'), ('numEpisodes', '160')]
[('cast', 'http://www.wikidata.org/entity/Q459638'), ('name', 'Stacy Edwards'), ('numEpisodes', '1')]
[('cast', 'http://www.wikidata.org/entity/Q706513'), ('name', 'Charles S. Dutton'), ('numEpisodes', '1')]
[('cast', 'http://www.wikidata.org/entity/Q5363239'), ('name', 'Elizabeth Morehead'), ('numEpisodes', '1')]
[('cast', 'http://www.wikidata.org/entity/Q64684485'), ('name', 'Caris Vujcec'), ('numEpisodes', '1')]
[('cast', 'http://www.wikidata.org/ent

## Task 4
Check who is the actor who acted in more films while working on "Without a Trace" (the result set must be a list of triples actor/actress IRI, label and #films).

In [143]:
queryString = """
SELECT DISTINCT ?cast ?name (COUNT(?name) AS ?NumFilms)
WHERE {
  
  #Get the list of cast memebers in Without A Trace
  wd:Q826477 wdt:P527 ?obj. #WithOutATrance has parts 
  #?obj sc:name ?season. 
  ?obj wdt:P527 ?ep. #has parts has episodes
  ?ep wdt:P161 ?cast. #episodes have cast member
  ?cast sc:name ?name.
  
  #Get the films names for the above cast members
  ?filmName wdt:P161 ?cast.
  ?filmName sc:name ?film.
  
  #Filtering all the films with intance as "film"
  ?filmName wdt:P31 ?o. #film has instance as object
  ?o sc:name ?instance.
  
  FILTER(str(?instance) = "film").
  
} GROUP BY ?cast ?name
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('cast', 'http://www.wikidata.org/entity/Q308124'), ('name', 'Anthony LaPaglia'), ('NumFilms', '7520')]
[('cast', 'http://www.wikidata.org/entity/Q459638'), ('name', 'Stacy Edwards'), ('NumFilms', '12')]
[('cast', 'http://www.wikidata.org/entity/Q706513'), ('name', 'Charles S. Dutton'), ('NumFilms', '39')]
[('cast', 'http://www.wikidata.org/entity/Q5363239'), ('name', 'Elizabeth Morehead'), ('NumFilms', '4')]
[('cast', 'http://www.wikidata.org/entity/Q235075'), ('name', 'Poppy Montgomery'), ('NumFilms', '800')]
[('cast', 'http://www.wikidata.org/entity/Q20651783'), ('name', 'Jenica Bergere'), ('NumFilms', '7')]
[('cast', 'http://www.wikidata.org/entity/Q503040'), ('name', 'Eric Close'), ('NumFilms', '640')]
[('cast', 'http://www.wikidata.org/entity/Q1721796'), ('name', 'Kaitlin Doubleday'), ('NumFilms', '3')]
[('cast', 'http://www.wikidata.org/entity/Q297128'), ('name', 'David Henrie'), ('NumFilms', '5')]
[('cast', 'http://www.wikidata.org/entity/Q3260118'), ('name', 'Lou Cute

Final query for this task

In [145]:
queryString = """
SELECT DISTINCT ?cast ?name (COUNT(?name) AS ?NumFilms)
WHERE {
  
  #Get the list of cast memebers in Without A Trace
  wd:Q826477 wdt:P527 ?obj.
  #?obj sc:name ?season. 
  ?obj wdt:P527 ?ep.
  ?ep wdt:P161 ?cast.
  ?cast sc:name ?name.
  
  #Get the films names for the above cast members
  ?filmName wdt:P161 ?cast.
  ?filmName sc:name ?film.
  
  #Filtering all the films with intance as "film"
  ?filmName wdt:P31 ?o. #film has instance as object
  ?o sc:name ?instance.
  
  FILTER(str(?instance) = "film").
  
} GROUP BY ?cast ?name
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('cast', 'http://www.wikidata.org/entity/Q308124'), ('name', 'Anthony LaPaglia'), ('NumFilms', '7520')]
[('cast', 'http://www.wikidata.org/entity/Q459638'), ('name', 'Stacy Edwards'), ('NumFilms', '12')]
[('cast', 'http://www.wikidata.org/entity/Q706513'), ('name', 'Charles S. Dutton'), ('NumFilms', '39')]
[('cast', 'http://www.wikidata.org/entity/Q5363239'), ('name', 'Elizabeth Morehead'), ('NumFilms', '4')]
[('cast', 'http://www.wikidata.org/entity/Q235075'), ('name', 'Poppy Montgomery'), ('NumFilms', '800')]
[('cast', 'http://www.wikidata.org/entity/Q20651783'), ('name', 'Jenica Bergere'), ('NumFilms', '7')]
[('cast', 'http://www.wikidata.org/entity/Q503040'), ('name', 'Eric Close'), ('NumFilms', '640')]
[('cast', 'http://www.wikidata.org/entity/Q1721796'), ('name', 'Kaitlin Doubleday'), ('NumFilms', '3')]
[('cast', 'http://www.wikidata.org/entity/Q297128'), ('name', 'David Henrie'), ('NumFilms', '5')]
[('cast', 'http://www.wikidata.org/entity/Q3260118'), ('name', 'Lou Cute

## Task 5
Compare Without a Trace with the tv series "Cold Case" in terms of number of seasons, episods and cast members (the result set must be two elements -one for each tv series- of tv series IRI, label, #seasons, #episodes and #cast members).

In [186]:
queryString = """
SELECT ?s ?name ?NumSeasons
WHERE {
  
  #Get the IRI of both the tv series
  ?s wdt:P31 wd:Q5398426.
  ?s sc:name ?name.
  
  FILTER(str(?name) = "Without a Trace" || str(?name) = "Cold Case").
  
 
} LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q733960'), ('name', 'Cold Case')]
[('s', 'http://www.wikidata.org/entity/Q826477'), ('name', 'Without a Trace')]
2


Final query for this task

In [191]:
queryString = """
SELECT ?IRI ?label ?seasons ?episodes (COUNT(?castName) AS ?castMembers)
WHERE {
  
  #Get the IRI and label of both the tv series
  ?IRI wdt:P31 wd:Q5398426.
  ?IRI sc:name ?label.
  
  FILTER(str(?label) = "Without a Trace" || str(?label) = "Cold Case").
  
  # Get the number of seasons
  ?IRI wdt:P2437 ?seasons.
  
  # Get the number of episodes
  ?IRI wdt:P1113 ?episodes.
  
  # Get the number of cast members
  ?IRI wdt:P161 ?cast.
  ?cast sc:name ?castName.
  
 
} GROUP BY ?IRI ?label ?seasons ?episodes
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('IRI', 'http://www.wikidata.org/entity/Q733960'), ('label', 'Cold Case'), ('seasons', '7'), ('episodes', '156'), ('castMembers', '478')]
[('IRI', 'http://www.wikidata.org/entity/Q826477'), ('label', 'Without a Trace'), ('seasons', '7'), ('episodes', '160'), ('castMembers', '2')]
2


## Task 6
Return the actors who are members of the cast of Without a Trace have [Kavin Bacon number](https://en.wikipedia.org/wiki/Six_Degrees_of_Kevin_Bacon#:~:text=Kevin%20Bacon%20himself%20has%20a,Bacon%20number%20is%20N%2B1.) equal to 2 (the result set must be a list of couples actor/actress IRI and label).

In [194]:
queryString = """
SELECT DISTINCT ?cast ?name
WHERE {
  
  #Get the list of cast memebers in Without A Trace
  wd:Q826477 wdt:P527 ?obj. #WithOutATrance has parts 
  #?obj sc:name ?season. 
  ?obj wdt:P527 ?ep. #has parts has episodes
  ?ep wdt:P161 ?cast. #episodes have cast member
  ?cast sc:name ?name.
  
  #Get the films names for the above cast members
  ?filmName wdt:P161 ?cast.
  ?filmName sc:name ?film.
  
} 
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('cast', 'http://www.wikidata.org/entity/Q373989'), ('name', 'David Paymer')]
[('cast', 'http://www.wikidata.org/entity/Q430872'), ('name', 'Marianne Jean-Baptiste')]
[('cast', 'http://www.wikidata.org/entity/Q139341'), ('name', 'Zach Grenier')]
[('cast', 'http://www.wikidata.org/entity/Q3265830'), ('name', 'Lucille Soong')]
[('cast', 'http://www.wikidata.org/entity/Q308124'), ('name', 'Anthony LaPaglia')]
[('cast', 'http://www.wikidata.org/entity/Q238501'), ('name', 'Alex Veadov')]
[('cast', 'http://www.wikidata.org/entity/Q706513'), ('name', 'Charles S. Dutton')]
[('cast', 'http://www.wikidata.org/entity/Q355163'), ('name', 'Bruce Davison')]
[('cast', 'http://www.wikidata.org/entity/Q1125651'), ('name', 'Thom Barry')]
[('cast', 'http://www.wikidata.org/entity/Q235075'), ('name', 'Poppy Montgomery')]
[('cast', 'http://www.wikidata.org/entity/Q1721796'), ('name', 'Kaitlin Doubleday')]
[('cast', 'http://www.wikidata.org/entity/Q445325'), ('name', 'Megan Gallagher')]
[('cast', '

Final query for this task

In [ ]:
#

## Task 7
Consider the actors who are members of the cast of Cold Case. Amongst the tv series which these actors acted return only those which received more than 2 awards (the result set must be triples of tv series IRI, label, #awards won).

In [98]:
queryString = """
SELECT DISTINCT ?name (Count(?won) AS ?numAward)
WHERE {
     #?award wdt:P31 ?obj.
     #?obj sc:name ?name.
     
     # 
     wd:Q733960 wdt:P161 ?obj.
     ?obj wdt:P166 ?award.
     ?award sc:name ?won.
     ?obj sc:name ?name.
     
     #FILTER(?won>"2").
     #FILTER REGEX((?name), "^award").
     
 
} GROUP BY ?name
HAVING(count(?won)>2)
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('name', 'Bobby Cannavale'), ('numAward', '4')]
[('name', 'Shailene Woodley'), ('numAward', '7')]
[('name', 'Ken Howard'), ('numAward', '3')]
[('name', 'Bill Clinton'), ('numAward', '26')]
[('name', 'June Lockhart'), ('numAward', '3')]
[('name', 'Jennifer Lawrence'), ('numAward', '42')]
[('name', 'Melissa Leo'), ('numAward', '3')]
7


Final query for this task

In [212]:
queryString = """
SELECT DISTINCT ?IRI ?label ?name (Count(?won) AS ?numAward)
WHERE {

      #Get the IRI and label of Cold Case
      ?IRI wdt:P31 wd:Q5398426.
      ?IRI sc:name ?label.
  
      FILTER(str(?label) = "Cold Case").
     
     
     # Get the cast of Cold Case
     wd:Q733960 wdt:P161 ?obj.
     ?obj sc:name ?name.
     
     ?obj wdt:P166 ?award.
     ?award sc:name ?won.
     
 
} GROUP BY ?IRI ?label ?name
HAVING(count(?won)>2)
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('IRI', 'http://www.wikidata.org/entity/Q733960'), ('label', 'Cold Case'), ('name', 'Jennifer Lawrence'), ('numAward', '42')]
[('IRI', 'http://www.wikidata.org/entity/Q733960'), ('label', 'Cold Case'), ('name', 'Bobby Cannavale'), ('numAward', '4')]
[('IRI', 'http://www.wikidata.org/entity/Q733960'), ('label', 'Cold Case'), ('name', 'Melissa Leo'), ('numAward', '3')]
[('IRI', 'http://www.wikidata.org/entity/Q733960'), ('label', 'Cold Case'), ('name', 'June Lockhart'), ('numAward', '3')]
[('IRI', 'http://www.wikidata.org/entity/Q733960'), ('label', 'Cold Case'), ('name', 'Shailene Woodley'), ('numAward', '7')]
[('IRI', 'http://www.wikidata.org/entity/Q733960'), ('label', 'Cold Case'), ('name', 'Bill Clinton'), ('numAward', '26')]
[('IRI', 'http://www.wikidata.org/entity/Q733960'), ('label', 'Cold Case'), ('name', 'Ken Howard'), ('numAward', '3')]
7
